In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import zipfile
import os
import shutil

In [ ]:
def extract_files(source_path, target_path):
    zip_ref = zipfile.ZipFile(source_path,'r')
    zip_ref.extractall(target_path)
    zip_ref.close()

In [ ]:
extract_files('/kaggle/input/dogs-vs-cats/test1.zip','/kaggle/working/')
extract_files('/kaggle/input/dogs-vs-cats/train.zip','/kaggle/working/')

In [ ]:
print(os.listdir('/kaggle/working/'))
print(os.listdir('/kaggle/working/train/')[:5])
print(os.listdir('/kaggle/working/test1/')[:5])

In [ ]:
#creating directories for cat and dog 
os.makedirs('/kaggle/working/training/')
os.makedirs('/kaggle/working/training/dog/')
os.makedirs('/kaggle/working/training/cat/')

os.listdir('/kaggle/working/training/')

In [ ]:
#Separating dogs and cats images in separate folder for training set
target_path = '/kaggle/working/training/'

def copy_file(path,file_name,target):
    source_path = os.path.join(path,file_name)
    destination_path = os.path.join(target_path,target)
    shutil.copy(source_path,destination_path) 

def segregate_images(path):
    for img in os.listdir(path):
        category = img.split(".")[0]
        if category == 'cat':
            copy_file(path,img,'cat')
        else:
            copy_file(path,img,'dog')

In [ ]:
segregate_images('/kaggle/working/train/')

In [ ]:
print("Total no of cat images :", len(os.listdir('/kaggle/working/training/cat')))
print("Total no of dog images :", len(os.listdir('/kaggle/working/training/dog')))

train_cat_names= os.listdir('/kaggle/working/training/cat')
train_dog_names = os.listdir('/kaggle/working/training/dog')

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

In [ ]:
fig = plt.gcf()
fig.set_size_inches(nrows*4,ncols*4)

pic_index+=8

cat_pic = [os.path.join('/kaggle/working/training/cat/',fname) 
          for fname in train_cat_names[pic_index-8:pic_index]]

dog_pic = [os.path.join('/kaggle/working/training/dog/',fname)
          for fname in train_dog_names[pic_index-8:pic_index]]

for i, img_path in enumerate(cat_pic+dog_pic):
    
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)

    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()

In [ ]:

model = tf.keras.Sequential([
    
    #Conv layer:1
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    #Conv layer:2
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Conv layer 3
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Conv layer 4
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    #Conv layer 5
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.MaxPool2D(2,2),
    #Flatten
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
    
])

model.summary()

In [ ]:
#Compile the model with loss='binary_crossentropy' and optimizer=RMSProp
model.compile(loss='binary_crossentropy', 
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-8),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1.0/255.0)

train_data_generator = train_gen.flow_from_directory(
    '/kaggle/working/training/',
    batch_size=250,
    target_size=(150,150),
    class_mode='binary'
)


In [ ]:
mycallback = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=10)
history = model.fit(train_data_generator, epochs=5, 
                    steps_per_epoch=5, 
                   verbose=1)

In [ ]:
model.save('cats_vs_dogs.h5')


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

path = '/kaggle/working/test1'
output = []
for file in os.listdir(path):
    k = {}
    test_path = os.path.join(path,file)
    k['file_path'] = test_path
    img = image.load_img(test_path,target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    
    classes = model.predict(images,batch_size=10)
    print(classes[0])
    if classes[0]> 0 :
        k['prediction']='dog'
    else:
        k['prediction']='cat'
    output.append(k)

print(k)

In [ ]:
import pandas as pd

out_data = pd.DataFrame(output)
out_data.head()

out_data.to_csv('/kaggle/working/predictions.csv',index=False)